<a href="https://colab.research.google.com/github/luisam19/course_optimizacion/blob/main/Dualidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='056938'> **Importar librerias** </font>

In [ ]:
!pip install -q amplpy
from amplpy import AMPL, tools
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import sympy as sp
from sympy.matrices import Matrix
from sympy.vector import Vector
import numpy as np

# <font color='056938'> **Problema base** </font>

Para la discusión de esta unidad usaremos como base un problema de mezcla

## <font color='8EC044'> **Descripción del problema** </font>

Una empresa produce cuatro tipos de productos: Producto A, Producto B, Producto C y Producto D. Cada producto es una mezcla de diferentes ingredientes, y la empresa desea maximizar su beneficio. El beneficio por unidad de cada producto es el siguiente:

| Producto | Bneficioe |
|-------------|---------------------|
| A           | 20        |
| B           | 15        |
| C           | 25        |
| D           | 50        |

El proceso de producción requiere los siguientes ingredientes por unidad de cada producto:

| Producto | Ingrediente 1 | Ingrediente 2 |Ingrediente 3 |
|----------|---------------|---------------|---------------|
| A        | 2             | 3             | 1             |
| B        | 1             | 2             | 2             |
| C        | 3             | 1             | 4             |
| D        | 2             | 4             | 3             |

La empresa cuenta con las siguientes cantidades de ingredientes disponibles:


| Ingrediente | Cantidad disponible |
|-------------|---------------------|
| 1           | 60 unidades        |
| 2           | 50 unidades        |
| 3           | 100 unidades        |


Además, la empresa quiere asegurarse de que el número de puntos por producción sostenible sea mayor al menos de 50 unidades. Donde los puntos asociados a cada unidad de producto son:

| Producto | puntos |
|-------------|---------------------|
| A           | 4        |
| B           | 3        |
| C           | 2        |
| D           | 1        |



## <font color='8EC044'> **Formulación del problema** </font>


\begin{align*}
\text{maximizar} \ z = &20x_A + 15x_B + 25x_C + 50x_D\\
\text{s.a.} \\
\text{Ingredientes 1:} & \quad 2x_A + x_B + 3x_C + 2x_D \leq 60 \\
\text{Ingredientes 2:} & \quad 3x_A + 2x_B + x_C + 4x_D \leq 50 \\
\text{Ingredientes 3:} & \quad x_A + 2x_B + 4x_C + 3x_D \leq 100 \\
\text{Producción sost.:} & \quad 4x_A + 3x_B + 2x_C + 1x_D \geq 50 \\
\text{No negatividad:} & \quad x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}

## <font color='8EC044'> **Implementación** </font>

En este caso usaremos `AMPL` para modelar el problema y `CPLEX` para resolverlo, puesto que de esta forma podemos obtener fácilmente la información que requeriremos en el analisis de sensibilidad. Dependiendo del modelador y del solver de solución puede ser posible o no obtener esta infromación fácilmente.

In [ ]:
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # Los optimizadores que vamos a usar
    license_uuid="default") # license to use (Aqui hay que poner su licencia :-;

En nuestro caso, crearemos la  función `create_model()` que genera el modelo para usarla en caso que tengamos que  generarlo para distintos conjuntos de datos.

In [ ]:
def create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min):
  #Conjuntos
  ampl.eval(
      """ reset;
          set Products;          #Conjunto de producos
          set Ingredients;				#Conjunto de ingredientes
          #Parameters
          param benefit{Products}>=0;              # Beneficio de cada producto
          param dispon{Ingredients}>=0;              # disponibilidad de cada ingrediente
          param requerim {Ingredients,Products}>=0; # Requerimiento de ingrediente por producto
          param volumn{Products}>=0;              # Aporte por unidad de producto
          param prod_min >=0;                             # Numero minimo de productos

          #Variables
          var x{j in Products}>=0;  # Cantidad de producto a fabricar

          #Objective function
          maximize TotalBenefit: sum{j in Products}benefit[j]*x[j];

          #Constraints
          subject to Disponibilidad{i in Ingredients}:
          sum{j in Products}requerim[i,j]*x[j]<=dispon[i];

          subject to ProdMin:
          sum{j in Products}volumn[j]*x[j]>=prod_min;""")

  # instantiate the model
  #Parametros cargados directamente
  ampl.set["Products"] = products
  ampl.set["Ingredients"] = ingredients
  ampl.param["benefit"] = benefits
  ampl.param["dispon"] = disponibility
  ampl.param["requerim"] = requirement
  ampl.param["volumn"] = volumn
  ampl.param["prod_min"] = prod_min

  return ampl


Construimos los datos de la instancia para alimentar la función que crea el modelo

In [ ]:
products = ["A", "B", "C", "D"]
ingredients = [1, 2, 3] #En Python contamos desde 0 ! Acuerdate de esto
benefits = {"A": 20, "B": 15, "C": 25, "D": 50}
disponibility = {1: 60, 2: 50, 3: 100}
requirement = {
    (1, 'A'): 2,
    (1, 'B'): 1,
    (1, 'C'): 3,
    (1, 'D'): 2,
    (2, 'A'): 3,
    (2, 'B'): 2,
    (2, 'C'): 1,
    (2, 'D'): 4,
    (3, 'A'): 1,
    (3, 'B'): 2,
    (3, 'C'): 4,
    (3, 'D'): 3
}
volumn = {"A": 4, "B": 3, "C": 2, "D": 1}
prod_min = 50



Para efectos de correr el modelo y poder evidenciar los conceptos del análisis de sensibilidad usaremos `cplex` desactivando las opciones de presolver y activando el componente de sensibilidad

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

Corremos el modelo e imprimimos la solución de las variables de decisión y el valor de la función objetivo

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Imprimimos los resultados
TotalBenefit = ampl.get_objective("TotalBenefit")
print("\nObjective is:", TotalBenefit.get().value())
ampl.get_data("x").to_pandas()

# <font color='056938'> **Introducción** </font>

Considere una función de dos variables $f : A × B ↦ C$ toma dos valores de entrada, $a ∈ A$ y $b ∈ B$, y produce un valor de salida $ f(a, b) = c ∈ C$.

> La función $f$ se puede ver como una colección de funciones, una para cada $a \in A $, que mapea $B$ a $C$. Es decir, considere $f$ como el conjunto de funciones $\{g_a : B ↦ C | a ∈ A\}$ , dónde $g_a(b) ≡ f(a, b) \ ∀a ∈ A, ∀b ∈ B$.

> Al intercambiar el rol de $A$ y $B$  se considera el conjunto de funciones $\{hb : A ↦ C | b ∈ B\}$ dónde $h_b(a) ≡ f(a, b) \ ∀b ∈ B, ∀a ∈ A$. \\

Estas dos formas de ver f son duales entre si. [1]

![](https://docs.google.com/uc?export=download&id=1MJt0Ts8UUIBFivv-VSDUb-XWRPJbi3zf)

Por ejemplo, $f(x, y) = 5x + x^2y - 10y^3 $ se puede ver como un conjunto de funciones de $y$, una para cada valor real de $x$.
> Para $x = 1$, $g_1(y) = 5 + y - 10y^3$.

> Para $x = 20$, $g_{20}(y) = 100 + 400y - 10y^3 $

De forma similar, puede verse $f$ como un conjunto de funciones de $x $, una para cada valor real de $y$.
> Para $y = 0$, $h_0(x) = 5x $.

> Para $y = -1$, $h_{-1}(x) = 5x - x^2 + 10 $.



## <font color='8EC044'> **Dualidad en el juego de Sudoku** </font>

El Sudoku es  una matriz de 9 × 9 que se subdivide en nueve submatrices de 3 × 3 mediante líneas entre las filas 3 y 4, las filas 6 y 7, las columnas 3 y 4, y las columnas 6 y 7. Cada celda de la matriz debe contener un entero entre 1 y 9. Inicialmente, algunas de las celdas contienen enteros y el resto están vacías. El objetivo del Sudoku es completar la matriz de manera que cada fila, columna y submatriz de 3 × 3 contenga todos los números del 1 al 9, sin que se repitan en ninguna de ellas.

- $M_{i,:}$: denota las columnas de la cuadrícula de Sudoku, donde $i = 1, 2, \ldots, 9$.
- $M_{:,j}$: denota las filas de la cuadrícula de Sudoku, donde $j = 1, 2, \ldots, 9$.
- $M\{i, i + 1, i + 2\}, \{j, j + 1, j + 2\}$: denota las submatrices de 3 × 3 dentro de la cuadrícula de Sudoku, donde $i = 1, 4, 7$ y $j = 1, 4, 7$.



Se define la solución a un Sudoku como una función $f : A \times B \rightarrow C \$ donde:

- $A = \{1, 2, 3, 4, 5, 6, 7, 8, 9\}$, el conjunto de posibles valores asignados a las celdas.
- $B = \{(i, j) : 1 \leq i \leq 9; 1 \leq j \leq 9\}$, el conjunto de celdas.
- $C = \{T, F\}$, representando "Verdadero" y "Falso".
- $f(a, b) = T$ si el valor $a$ está asignado a la celda $b$; $f(a, b) = F$ si no lo está.

> Para cada entero $a \in A$, la función $g_a : B \rightarrow C$ indica a qué celdas se asigna $a$.

> Para cada celda $b \in B$, la función $h_b : A \rightarrow C$ indica qué entero se asigna a $b$.

Estos dos conjuntos de funciones son duales entre sí.

### <font color='46B8A9'> **Ejercicio** </font>

Considere el siguiente Sudoku

![](https://docs.google.com/uc?export=download&id=146qHYwpk-njXEQ3YBEf0Sr_Kyd4y_cg8)

> ¿Dónde debe ubicarse el 2 en la submatriz central \(M\{4,5,6\},\{4,5,6\}\)?

> ¿Qué valor debe asignarse a la celda (1, 5)?



# <font color='056938'> **Dualidad en programación lineal** </font>

Para cada programa lineal que resolvemos, hay otro programa lineal asociado  que estamos resolviendo simultáneamente. Este nuevo programa lineal cumple algunas propiedades  importantes. Puede ser utilizado para obtener la solución al programa original, y sus variables proporcionan información útil sobre el conjunto de soluciones óptimas para el programa lineal original.

En el programa lineal asociado definimos nuevas variables de sensibilidad y formulamos el programa lineal asociado que nos permita obtener los valores que deseamos conocer. Para ello será necesario diferenciar entre los dos problemas.

> El problema **primal** es el modelo de optimización dado, aquel que formula la aplicación de interés primario.

> El problema **dual** correspondiente a una formulación primal dada es un programa lineal estrechamente relacionado, con variables de decisión y restricciones que cuantifican la sensibilidad de los resultados primales a cambios en sus parámetros.

Construiremos cada uno de los componentes del problema **dual**.



## <font color='8EC044'> **Variables duales** </font>

Las variables duales cuantificaran el cambio en la función objetivo ante cambios en los valores del lado derecho (vector de recursos) de las restricciones.

Note que para para el problema de mezcla que usamos de referencia, el valor de dicho cambio se visualiza como la pendiente de la curva en diferentes valores del parámetro.

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("dispon")
# iterate over different values of the parameter
for value in range(0, 200):
  change_param.set_values({1: value})
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

> Hay una **variable dual** (que denotaremos $v_i$) para cada restricción $i$ del primal. Cada una refleja la tasa de cambio en el valor óptimo del primal por cada unidad de aumento desde el valor dado del lado derecho de la correspondiente restricción.


Dado que el **dual** solo proporciona un análisis subsidiario del primal, que es el modelo de interés real. Los valores óptimos de las variables duales para cualquier PL  solo generan tasas de cambio en el valor original del parámetro del lado derecho

<font color='8EC044'> **Tipo de variable dual** </font>

Considerando el análisis que ya hicimos sobre relajar o hacer más estricto el programa lineal nos permite determinar el conjunto de valores o tipo de variable dual que debemos esperar. Por ejemplo:

> Un aumento en el lado derecho de una restricción $\leq$ en un modelo de maximización relaja la restricción, lo que deja el valor óptimo sin cambios o reducido. Por lo que la variable dual correspondiente $v_i \geq 0$.








### <font color='46B8A9'> **Ejercicio** </font>

Analice el cambio resultante de aumentar en una unidad el lado derecho de una restricción y complete la siguiente tabla:

|     Primal |Restricción $\leq$|Restricción $\geq$|Restricción $=$|
|------------|------------------|------------------|---------------|
|Maximización|                  |                  |No restringida |
|Minimización|   $v_i \geq 0$   |                  |               |


Note por ejemplo que para el caso de la restricción de producción sostenible (restricción $\geq$) el el tipo de variable dual es $v \leq 0$. Es decir, la pendiente es negativa.

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("prod_min")
print(change_param)
# iterate over different values of the parameter
for value in range(0, 200):
  change_param.set_values([value])
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

<font color='8EC044'> **Interpretación económica** </font>

Debido a que las variables duales nos indican la tasa de cambio en el valor óptimo para una unidad adicional de cada lado derecho (RHS, por sus siglas en inglés), proporcionan una especie de precio implícito sobre el recurso de cada restricción del modelo. Para ser más precisos, producen lo que los economistas llaman **precios marginales**.

### <font color='46B8A9'> **Ejercicio** </font>

En el problema de mezcla, estime e **interprete** el precio marginal de disponer de una unidad extra del ingrediente 2. Use para ello la gráfica del valor de la función objetivo respecto al lado derecho y estime manualmente la pendiente

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("dispon")
# iterate over different values of the parameter
for value in range(0, 200):
  change_param.set_values({2: value})
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

Repita el ejercicio anterior, esta vez considerando la restricción asociada al ingrediente 3

In [ ]:
# Escriba aquí su respuesta

## <font color='8EC044'> **Restricciones del problema dual** </font>

La actividad asociada a cualquier variable de decisión consume recursos (en una restricción de $\leq$) o crean recursos (en una restricción de $\geq$) de las restricciones




<font color='8EC044'>**Costo o precio implícito de una actividad** </font>

Los coeficientes de una actividad (variable) en cada restricción  pueden interpretarse como cantidades consumidas o creadas por unidad de la actividad. Al sumar esos coeficientes multiplicados por el precio implícito $(v_i)$ de los recursos involucrados, podemos obtener un valor marginal de la actividad, en problemas de minimización, o un precio  de la actividad en problemas de maximización.

> El valor marginal implícito (en problemas de minimización) o el precio (en problemas de maximización) de una unidad de actividad (variable primal) $j$ generada por los valores de las variables duales $v_i$ es $\sum_{i} a_{ij}v_i$, donde $a_{ij}$ denota el coeficiente de la actividad $j$ en el lado izquierdo de la restricción $i$.

En el problema de mezcla, el costo marginal de una unidad de producto $A$ estaria dado por:
> $\sum_{i=1}^4 a_{iA}v_i = 2v_1 + 3v_2 + 1v_3+ 4v_4$

### <font color='46B8A9'> **Ejercicio** </font>

Determine como se calcula el costo marginal implícito de una unidad de producto D en términos de las variables duales.

<font color='8EC044'>**Restricción para garantizar costo o precio de las actividades** </font>


El valor implícito de la actividad por unidad permite construir las restricciones duales. Si las variables duales $v_i$ realmente  reflejar el valor de los recursos de la restricción en el valor óptimo primal, el valor implícito de la actividad debe ser consistente con los costos o beneficios unitarios explícitos en la función objetivo primal.

> Para cada variable no negativa de actividad $x_j$ en un programa lineal hay una correspondiente restricción dual
* En un programa de minimización, la  restricción dual $\sum_ia_{ij}v_i \leq c_j$  asegura que el valor marginal neto de la actividad no exceda su costo dado.
* En un problema de maximización, la restricción dual $\sum_ia_{ij}v_i \geq c_j$, mantiene el costo marginal neto de cada actividad al menos igual a su beneficio dado.

En el problema de mezcla, la restricción del costo marginal de una unidad de producto $A$ estaria dada por:
> $\sum_{i=1}^4 a_{iA}v_i \geq c_A \longrightarrow 2v_1 + 3v_2 + 1v_3+ 4v_4 \geq 20$

### <font color='46B8A9'> **Ejercicio** </font>

Determine la restricción dual asociada al costo marginal implícito de una unidad de producto D en términos de las variables duales.

## <font color='8EC044'> **Función objetivo del problema dual** </font>

La función objetivo del problema dual tiene un sentido opuesto a la del problema primal. Es decir para un problema primal de maximización el dual es de minimización,  mientras que para un problema primal de minimización el problema duale es de maximización.

> Para un programa lineal la función objetivo considera el valor total de los recursos disponibles o los requeridos. Es decir,
* Para un problema primal de maximización, el problema dual minimiza el valor implicito total de los recursos disponibles estableciendo precios justos para cada uno de ellos, es decir, $\min \sum_{i} b_iv_i$
* Para un problema de minimización, el problema dual minimiza el valor costo total de los recursos requeridos, es decir, $\max \sum_{i} b_iv_i$

### <font color='46B8A9'> **Ejercicio** </font>

Determine la función objetivo para el problema de mezcla que hemos venido discutiendo en este notebook

# <font color='056938'> **Interpretación económica del problema dual** </font>

Para tener una visión completa del problema lineal, es de utilidad revisar la interpretación del economica del problema dual completo [2]

## <font color='8EC044'> **Variables duales** </font>

Consideremos el siguiente programa lineal :


\begin{align*}
& \text{Minimizar } \mathbf{c}^\top \mathbf{x} \\
& \text{sujeto a } \mathbf{Ax} \geq \mathbf{b} \\
& \mathbf{x} \geq \mathbf{0}
\end{align*}

y su dual


\begin{align*}
& \text{Maximizar } \mathbf{v}^\top \mathbf{b} \\
& \text{sujeto a } \mathbf{vA} \leq \mathbf{c} \\
& \mathbf{v} \geq \mathbf{0}
\end{align*}



Sea $B$ una base óptima para el problema primal con $\mathbf{c}_B$ como su vector de costos asociado. Supongamos que esta solución básica factible óptima $x^*$ es no degenerada. Denotando $J$ como el conjunto de índices para las variables no básicas $x_j$, que pueden incluir tanto variables de holgura y execeso como variables estructurales (es decir, $J \subseteq \{1, \ldots, n + m\}$, tenemos:

> $z = \mathbf{c}_B^T \mathbf{b} - \sum_{j \in J} (c_j - \mathbf{c}_B^T \mathbf{B}^{-1} \mathbf{A}_j) x_j = \mathbf{v}^T \mathbf{b} - \sum_{j \in J} (\mathbf{v} - \mathbf{c}_j) x_j. \quad $

Si el i-ésimo lado derecho $b_i$ se perturba ligeramente (positiva o negativamente), dado que la solución factible básica actual permanece factible, mantenemos su optimalidad. Por lo tanto, si $z$ es el valor óptimo de la función objetivo y $\mathbf{B}_i$ es la  i-ésima columna de $\mathbf{B}^{-1}$, tenemos:

> $\frac{\partial z^*}{\partial b_i} = \mathbf{c}_B^T \mathbf{B}_i = \mathbf{v}^*$


Así, $\mathbf{v}^*$ es la tasa de cambio del valor óptimo de la función objetivo con un aumento unitario en el valor del lado derecho $b_i$

### <font color='46B8A9'> **Ejercicio** </font>

Considere el programa lineal

\begin{align*}
\text{maximizar} \ z = &20x_A + 15x_B + 25x_C + 50x_D\\
\text{s.a.} \\
\text{Ingredientes 1:} & \quad 2x_A + x_B + 3x_C + 2x_D \leq 60 \\
\text{Ingredientes 2:} & \quad 3x_A + 2x_B + x_C + 4x_D \leq 50 \\
\text{Ingredientes 3:} & \quad x_A + 2x_B + 4x_C + 3x_D \leq 100 \\
\text{Producción sost.:} & \quad 4x_A + 3x_B + 2x_C + 1x_D \geq 50 \\
\text{No negatividad:} & \quad x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}

cuya solución óptima es la siguiente:

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Imprimimos los resultados
TotalBenefit = ampl.get_objective("TotalBenefit")
print("\nObjective is:", TotalBenefit.get().value())
ampl.get_data("x").to_pandas()


In [ ]:
ampl.display('_conname, _con.slack, _con.down,  _con.current, _con.up')

Teniendo en cuenta esta información obtenga el valor de las varaibles duales como
> $\frac{\partial z^*}{\partial b_i} = \mathbf{c}_B^T \mathbf{B}_i = \mathbf{v}^*$

In [ ]:
# Escriba aqui su respuesta



**tip**: Puede serle de utilidad como calcular la inversa de de una matriz y realizar el producto de un vector con una matriz

In [ ]:
from sympy.matrices import Matrix
from sympy.vector import Vector
B = Matrix(( [1, 2, 3], [3, 6, 2], [2, 0, 1] ))
cB = Matrix(( [1, 2, 3]))

# Calcular inversa
Binv = B.inv(method="LU")

# Efectuar producto
cB.transpose()*Binv



## <font color='8EC044'> **Problema de minimización** </font>

Supongamos que contratamos a una empresa para producir cantidades especificadas $b_1, b_2, ..., b_m$ de $m$ productos o bienes. La empresa puede participar en cualquiera de $n$ actividades a diferentes niveles para producir los productos. Cada actividad $j$ tiene su propio costo unitario $c_j$, y acordamos pagar el costo total de producción. Desde nuestro punto de vista, nos gustaría tener control sobre las operaciones de la empresa para que podamos especificar la combinación y niveles de actividades en los que la empresa participará para minimizar el costo total de producción. Si $a_{ij}$ denota la cantidad de producto $i$ generado por una unidad de actividad $j$, entonces $\sum_{j=1}^{n} a_{ij} x_j $ representa las unidades de producción $i$ que se producen. Estas deben ser mayores o iguales a la cantidad requerida $b_i$. Por lo tanto, deseamos resolver el siguiente problema, que es precisamente el problema primal:

\begin{align*}
\text{Minimizar:} & \quad \sum_{j=1}^{n} c_j  x_j \\
\text{Sujeto a:} & \quad \sum_{j=1}^{n} a_{ij} \cdot x_j \geq b_i \quad \text{para} \quad i = 1, 2, ..., m \\
& \quad x_j \geq 0 \quad \text{para} \quad j = 1, 2, ..., n
\end{align*}



En lugar de intentar controlar la operación de la empresa para obtener la mezcla más deseable de actividades, supongamos que acordamos pagar a la empresa precios unitarios $v_1, v_2, ..., v_m$ por cada uno de los $m$ productos. Sin embargo, también estipulamos que estos precios anunciados por la empresa deben ser justos. Dado que $a_{ij}$ es la cantidad de unidades del producto $i$ producidas por una unidad de actividad $j$ y $v_i$ es el precio unitario del producto $i$, entonces $\sum_{i=1}^{m} a_{ij} w_i $ puede interpretarse como el precio unitario de la actividad $j $ consistente con los precios $v_1, v_2, ..., v_m$. Por lo tanto, le decimos a la empresa que el precio implícito de la actividad $j$, es decir, $\sum_{i=1}^{m} a_{ij} v_i$, no debe exceder el costo real $c_j$. Por lo tanto, la empresa debe observar las restricciones $\sum_{i=1}^{m} a_{ij} v_i \leq c_j $ para $j = 1, 2, ..., n$ . Dentro de estas restricciones, la empresa desea elegir un conjunto de precios que maximice su retorno $\sum_{j=1}^{n} v_j b_j$. Esto conduce al siguiente problema dual considerado por la empresa:

\begin{align*}
\text{Maximizar:} & \quad \sum_{i=1}^{m} v_i  b_i \\
\text{s.a:} & \quad \sum_{i=1}^{m} a_{ij}  v_i \leq c_j \quad \text{para} \quad j = 1, 2, ..., n \\
& \quad v_i > 0 \quad \text{para} \quad i = 1, 2, ..., m
\end{align*}

## <font color='8EC044'> **Problema de maximización** </font>



Supongamos que el programa lineal es un problema de mezcla de productos en el que se están fabricando $n$ productos utilizando algunos $m$ tipos de recursos. Aquí, $x_j$ representa la cantidad de unidades producidas del producto $j$, y $b_i$ representa la cantidad de unidades disponibles para el recurso $i$ para $j = 1, ..., n$ e $i = 1, ..., m$. Una unidad del producto $j$ genera una ganancia de $c_j$ unidades y consume $a_{ij}$ unidades del recurso $i$ para $i = 1, ..., m $. Por lo tanto, deseamos resolver el siguiente problema, que es precisamente el problema primal:

\begin{align*}
\text{Maximizar:} & \quad \sum_{j=1}^{n} c_j x_j \\
\text{s.a.} & \quad \sum_{j=1}^{n} a_{ij}  x_j \leq b_i \quad \text{for} \quad i = 1, 2, ..., m \\
& \quad x_j \geq 0 \quad \text{for} \quad j = 1, 2, ..., n
\end{align*}


 Consideramos $w_i$ como el precio sombra, el valor implicito o el precio de mercado por una unidad de recurso $i$. Entonces, las restricciones duales tienen la siguiente interpretación. Observe que si el fabricante considera alquilar los $m$ recursos a precios unitarios $w_1, ..., w_m$ en lugar de fabricar la mezcla de productos, entonces cada unidad del producto no fabricada resultaría en una pérdida de ganancia de $c_i$ unidades, pero ganaría un alquiler de $\sum_{i=1}^m a_{ij} w_i$ unidades debido a los recursos que esto libera. Las restricciones duales $\sum_{i=1}^m a_{ij} w_i \geq c_j$ garantizan que esto sea al menos tan rentable como la fabricación de los productos. Sin embargo, para mantener precios justos o competitivos mientras se garantiza esta rentabilidad, el objetivo dual busca minimizar el alquiler total. Dando como resultado el siguiente problema dual:

 \begin{align*}
\text{Minimizar:} & \quad \sum_{j=1}^{m} b_j  v_j \\
\text{s.a.} & \quad \sum_{j=1}^{m} a_{ij} v_j \geq c_i \quad \text{for} \quad i = 1, 2, ..., n \\
& \quad v_j \geq 0 \quad \text{for} \quad j = 1, 2, ..., m
\end{align*}

# <font color='056938'> **Relaciones entre componentes del problema primal y dual** </font>

## <font color='8EC044'> **Igualdad entre el valor óptimo primal y dual** </font>
Si las variables duales están destinadas a fijar correctamente los precios de los recursos asociados con las restricciones, los suministros y las demandas de esos recursos deben equivaler exactamente al valor óptimo primal.

> Si un programa lineal primal tiene una solución óptima, su valor óptimo $\sum_{j=1}^{n} c_j  x_j^*$ es igual al valor total implícito dual óptimo $\sum_{i=1}^{m} b_i \cdot v_i^*$ de todos los recursos de restricción correspondientes.

En el caso del problema de mezcla tendriamos:

> $20x_A^* + 15x_B^* + 25x_C^* + 50x_D^* = 60v_1^* + 50v_2^* + 100v_3^* + 50v_4^*$

## <font color='8EC044'> **Holgura complementaria entre restricciones primales y variables duales** </font>

Una desigualdad está activa si se satisface como una igualdad por una solución dada, y está inactiva de lo contrario. Si una desigualdad está inactiva en la solución óptima primal, cambios pequeños en su lado derecho no afectarían en absoluto al valor óptimo; la restricción tiene holgura. Podemos deducir inmediatamente el valor de la variable dual correspondiente $v_i$.

> En la solución óptima primal, o bien la restricción principal de desigualdad $i$ está activa, o la variable dual correspondiente $v_i = 0$.

### <font color='46B8A9'> **Ejercicio** </font>

Considere la solución óptima del problema dual

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
ampl.display('_conname, _con.slack, _con.down,  _con.current, _con.up')

Obtenga e interprete el valor de la variable dual asociado a la tercera restricción $v_3$

## <font color='8EC044'> **Holgura complementaria entre variables primales y restricciones duales** </font>

Las restricciones del problema dual mantienen los precios de las actividades por debajo del costo real en problemas de minimización y por encima del beneficio real en problemas de maximización. A primera vista, podría parecer razonable pedir más (es decir, exigir que los precios de las actividades coincidan exactamente con los correspondientes $c_j$).

Habría una dificultad matemática práctica  en el caso común donde tenemos muchas más variables primales que restricciones. Relativamente pocas $v_i$ tendrían que satisfacer simultáneamente restricciones

 > $\sum_i a_{ij}  x_i = c_j$

para demasiadas actividades $j$.

Más importante aún, queremos que las variables duales midan el valor de los recursos en la optimalidad. Las únicas variables primales (no negativas) involucradas en una solución óptima son aquellas con $x_j^* > 0$. Limitar la valoración perfecta a esta lista más limitada de actividades produce las condiciones de holgura complementaria dual.

> O bien una variable primal no negativa tiene un valor óptimo $x_j = 0$, o los precios duales correspondientes $v_i$ deben hacer que la restricción dual La restricción dual $\sum_i a_i v_i \leq c_j $ o  $\sum_i a_i v_i \geq c_j$ sea activa.

# <font color='056938'> **Problema dual para problemas mixtos** </font>

En la práctica, muchos programas lineales contienen algunas restricciones del tipo $\leq$, algunas del tipo $\geq$ y algunas del tipo $=$. Además, las variables pueden ser $\geq 0$, $\leq 0$ o "no restringidas". En teoría, esto no presenta ningún problema ya que podemos aplicar las técnicas de transformación para convertir cualquier problema "mixto" en una de las formas canónicas, después de lo cual el dual se puede obtener fácilmente.

Sin embargo, tales conversiones pueden ser tediosas. Afortunadamente, no es necesario hacer estas conversiones realmente, y es posible declarar inmediatamente el dual de cualquier programa lineal, teniendo en cuenta para ello la siguiente tabla tomada de [2]:

![](https://docs.google.com/uc?export=download&id=1031YF3uKeTSnhDhK7R8mtg8VfK9Kchx0)


Por ejemplo, considere el siguiente problema primal


> $
\begin{align*}
\text{Maximizar:} \quad & 3x_1 - 2x_2 + x_3 \\
\text{s.a:} \quad & 2x_1 - x_2 + x_3 \geq 5 \\
& x_1 + 3x_2 - 2x_3 \leq 10 \\
& -x_1 + x_2 + 2x_3 < 7 \\
& x_1 + 2x_2 - 3x_3 = 12 \\
&x_1 \geq 0, \ x_2 \leq 0, \ x_3 \quad \text{ unrestricted}
\end{align*}
$

Usando la tabla anterior su dual sería:



> $
\begin{align*}
\text{Minimizar:} \quad & 5v_1 + 10v_2 + 7v_3 + 12v_4 \\
\text{s.a:} \quad & 2v_1 + v_2 - v_3 + v_4 \geq 3 \\
& -v_1 + 3v_2 + v_3 + 2v_4 \leq -2 \\
& v_1 + 2v_2 - 2v_3 \geq 1 \\
& 3v_1 - 2v_2 + 2v_3 \leq 1 \\
& v_1, v_2, v_3 \geq 0,  \ v_4 \ \text{ unrestricted}
\end{align*}
$



### <font color='46B8A9'> **Ejercicio** </font>

Cosntruya el problema dual asociado al siguiente problema primal

> $
\begin{align*}
\text{Maximizar:} \quad & 2x_1 - 3x_2 + 4x_3 \\
\text{s.a:} \quad & 2x_1 + x_2 - 3x_3 \leq 10 \\
& x_1 + 2x_2 - 3x_3 = 12 \\
& -x_1 + 2x_2 + 4x_3 < 7 \\
& 3x_1 - 2x_2 + x_3 \geq 5 \\
& x_1 - x_2 + 2x_3 = 8 \\
\quad & x_1 \leq 0, \ x_2 \geq 0, \ x_3, x_4 \text{ unrestricted}
\end{align*}
$



# <font color='056938'> **Resultados computacionales** </font>

al igual que encontramos el análisis de sensibilidad para las restricciones, el valor de la variable dual asociada a cada restricción puede calcularse directamente en el software

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Imprimimos los resultados
ampl.display('_conname, _con.slack, _con.down,  _con.current, _con.up, _con.dual')

# <font color='056938'> **Dualidad y optimalidad** </font>


Consideremos el siguiente programa lineal :


\begin{align*}
& \text{Minimizar } \mathbf{c}^\top \mathbf{x} \\
& \text{sujeto a } \mathbf{Ax} \geq \mathbf{b} \\
& \mathbf{x} \geq \mathbf{0}
\end{align*}

y su dual


\begin{align*}
& \text{Maximizar } \mathbf{v}^\top \mathbf{b} \\
& \text{sujeto a } \mathbf{vA} \leq \mathbf{c} \\
& \mathbf{v} \geq \mathbf{0}
\end{align*}


## <font color='8EC044'> **Dual del Dual** </font>

 Una de las relaciones más fáciles de ver.

> El dual del dual de cualquier programa lineal es el primal.

## <font color='8EC044'> **Teorema de dualidad débil entre funciones objetivo** </font>

los valores objetivos de las soluciones factibles de uno de los problemas acotan los valores de las soluciones factibles del otro.
> Respecto a la La función objetivo primal evaluada en cualquier solución factible
* Para un problema primal de minimización es mayor o igual que el valor de la función objetivo correspondiente del dual evaluada en cualquier solución factible del dual.
* Para un problema primal de maximización es  menor o igual que el valor de la función objetivo correspondiente del dual evaluada en cualquier solución factible del dual.

### <font color='46B8A9'> **Ejercicio** </font>



Considere el siguiente problema primal

\begin{align*}
\text{maximizar} \ z = &20x_A + 15x_B + 25x_C + 50x_D\\
\text{s.a.} \\
 &  2x_A + x_B + 3x_C + 2x_D \leq 60 \\
 &  3x_A + 2x_B + x_C + 4x_D \leq 50 \\
 & x_A + 2x_B + 4x_C + 3x_D \leq 100 \\
 & 4x_A + 3x_B + 2x_C + 1x_D \geq 50 \\
 & x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}


Cuyo dual es:



\begin{align*}
\text{minimizar} \ v = &60v_1 + 50v_2 + 100v_3 - 50v_4\\
\text{s.a.} \\
 &  2v_1 + 3v_2 + v_3 - 4v_4 \geq 20 \\
 &  v_1 + 2v_2 + 2v_3 - 3v_4 \geq 15 \\
 &  3v_1 + v_2 + 4v_3 - 2v_4 \geq 25 \\
 &  2v_1 + 4v_2 + 3v_3 - v_4 \geq 50 \\
 & v_1 \geq 0, \, v_2 \geq 0, \, v_3 \geq 0, \, v_4 \geq 0
\end{align*}

Considere las siguientes soluciones para el problema dual y el problema primal:


> $x_A = 10, x_B = 10, x_C = 10, x_D = 10$.

> $v_1 = 5, v_2 = 10, v_3 = 10, v_4 = 5$.




1. Verifique que las soluciones dadas son factibles en sus respectivos problemas
2. Valide el cumplimiento del teorema de dualidad débil

## <font color='8EC044'> **Infactibilidad o no acotamiento** </font>

> Si el programa lineal primal o su dual es no acotado, el problema asociado (dual o primal) es no factible

## <font color='8EC044'> **Holgura complementario y optimalidad** </font>

La holgura complementaria relaciona la holgura en las desigualdades del primal con valores no nulos de las variables duales correspondientes y viceversa. Otra mirada al cálculo de la dualidad débil (6.10) revela más. Las dos expresiones finales (no negativas) en la diferencia entre los valores de soluciones factibles del primal y el dual allí son exactamente la holgura complementaria total.

> Si $x$ es una solución factible para un programa lineal primal, y $v$ es una solución factible para el correspondiente dual, entonces la diferencia en sus valores objetivos es exactamente la holgura complementaria total entre las soluciones.
* $(c_j - vA)x = \sum_j \left(c_j- \sum_i v_i a_{ij}\right)x_j$
* $v(Ax -b) = \sum_i v_i \left(\sum_j a_{ij}x_j-b_i\right)$

> Además, si los valores de las funciones objetivo coinciden, es decir, $c^Tx = v^T b$, entonces tanto $x$ como $v$ son óptimos en sus respectivos problemas, y las condiciones de holgura complementaria entre las soluciones óptimas deben cumplirse en la optimalidad
* $(c-vA)x=0$
* $v(Ax - b)=0$


## <font color='8EC044'> **Teorema de dualidad fuerte** </font>

Las relaciones de dualidad débil entre los valores objetivos de soluciones factibles del primal y el dual establecen cotas para sus valores. Pero cuando ambos problemas tienen soluciones factibles óptimas, la  dualidad fuerte puede establecerse.

> Si un programa lineal primal o su dual tienen una solución óptima finita, ambos la tienen, y sus valores óptimos son iguales.






### <font color='46B8A9'> **Ejercicio** </font>

Considere el siguiente problema primal

\begin{align*}
\text{maximizar} \ z = &20x_A + 15x_B + 25x_C + 50x_D\\
\text{s.a.} \\
 &  2x_A + x_B + 3x_C + 2x_D \leq 60 \\
 &  3x_A + 2x_B + x_C + 4x_D \leq 50 \\
 & x_A + 2x_B + 4x_C + 3x_D \leq 100 \\
 & 4x_A + 3x_B + 2x_C + 1x_D \geq 50 \\
 & x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}


Cuyo dual es:



\begin{align*}
\text{minimizar} \ v = &60v_1 + 50v_2 + 100v_3 - 50v_4\\
\text{s.a.} \\
 &  2v_1 + 3v_2 + v_3 - 4v_4 \geq 20 \\
 &  v_1 + 2v_2 + 2v_3 - 3v_4 \geq 15 \\
 &  3v_1 + v_2 + 4v_3 - 2v_4 \geq 25 \\
 &  2v_1 + 4v_2 + 3v_3 - v_4 \geq 50 \\
 & v_1 \geq 0, \, v_2 \geq 0, \, v_3 \geq 0, \, v_4 \geq 0
\end{align*}

Teniendo en cuanta la solución óptima de ambos problemas, obtenida computacionalmente a continuación, Verifique el cumplimiento del teorema de dualidad Fuerte

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Imprimimos solución primal
print(ampl.get_data("x").to_pandas())
# Imprimimos solución dual
ampl.display(' _con.dual')

## <font color='8EC044'> **Condiciones de Karush–Kuhn–Tucker (KKT)** </font>

Adicionalmente, suponiendo que tenemos el siguiente problema de optimización lineal:



\begin{align*}
\text{Minimizar } & c^\top x \\
\text{sujeto a } & Ax \leq b \\
& x \geq 0
\end{align*}


Las condiciones de Karush-Kuhn-Tucker (KKT) para la optimalidad de una solución óptima $x$ y su dual asociado $v$  son:

1. **Factibilidad primal**: Las variables de decisión $x$ deben ser no negativas y satisfacer todas las restricciones de desigualdad: $x \geq 0$ y $Ax \leq b$.

2. **Factibilidad dual**: Las variables duales asociadas a las restricciones de desigualdad deben ser no negativas: $v \geq 0$. y $v^TA \leq c^T$

3. **Condiciones de complementariedad**: Para cada variable de decisión del primal o restricción del dual se satisface:
* $v_i(A_i^Tx -b_i) = 0 \quad \forall i = 1 \ldots m$
* $(c_j-v^Ta_j)x_j = 0 \quad \forall j = 1 \ldots n$

Esta última condición implica que:
* La varabiable dual asociada a una restricción debe ser cero cuando la variable de holgura o exceso de dicha restricción no es cero
* El costo reducido asociado a una variable debe ser cero cuando esta variable no sea cero

# <font color='056938'> **Práctica** </font>

La compañia **Nutricol** esta interesado en desrrollar una app que permita al usuario crear una dieta balanceada con base en el lstado de posibles ingredientes y requerimientos nutricionales específicos. Para ello ha contratado nuestros servicios para desrrollar un modelo que encuentre una dieta diaria de mínimo costo



## <font color='8EC044'> **Verbalización del modelo** </font>
Consideraremos una versión bastante simplificada del problema. Para ello definamos los elementos



#### <font color='46B8A9'> **Conjuntos** </font>

$I$: conjunto de ingredientes

$N$ conjunto de nutrientes

Note que el conjunto de ingredientes podria a su ve subdividirse en tipos de ingredientes como carnes, verduras, granos y frutas.

#### <font color='46B8A9'> **Parámetros** </font>

Debemos cosiderar los siguientes parametros

* costo por kilo de cada ingreiente
* aporte nutricional (en cada nutriente) de cada ingrediente
* catindad mínima de cada tipo de nutriente
* cantidad máxima de cada tipo de nutriente
* Debe existir proporcionalidad entre cada tipo de nutrientes (ej, la carne no debe excer más del 30% del menú)

#### <font color='46B8A9'> **Decisiones** </font>
Debemos decidir la cantidad de cada alimento que debe incluirse en el menú diario

#### <font color='46B8A9'> **Restricciones** </font>
La decisión esta sujeta a que:
* se cumplan los requisitos mínimos de cada nutriente
* se cumplan los requisitos máximos de cada nutriente
* No se excedan las cantidades máximas por tipos de nutrientes


#### <font color='46B8A9'> **Objetivo** </font>
Minimizar el costo total del menú


| Variable | Descripción                    |
|----------|--------------------------------|
| $I$      | Conjunto de ingredientes       |
| $S$      | Conjunto de tipos de ingredientes  |
| $N$      | Conjunto de nutrientes         |


| Parámetro | Descripción |
|-----------|-------------|
| $c_i$     | Costo por kilogramo del ingrediente $i \in I$. |
| $n_{ij}$  | Aporte nutricional del ingrediente $i \in I$ al nutriente $j \in N$. |
| $amin_j$   | Cantidad mínima requerida del nutriente $j \in N$. |
| $amax_j$   | Cantidad máxima permitida del nutriente $j \in N$. |
| $p_{k}$  | Proporción máxima de tipo de ingrediente $k$ en la mezcla. |
| $s_{ik}$  | Indicador binario que define si el ingrediente $i$ es del tipo $k$ |

La variable de decisión
> $x_i$: Cantidad de kilogramos del ingrediente $i \in I$ a incluir en el menú diario.




## <font color='8EC044'> **Formulación del modelo** </font>


\begin{align}
  \text{Minimizar} \quad \sum_{i \in I} c_i x_i\\
  \text{s.a.}\\
    \sum_{i \in I} n_{ij} x_i &\geq \text{amin}_j & \forall j \in N \\
    \sum_{i \in I} n_{ij} x_i &\leq \text{amax}_j & \forall j \in N \\
    \frac{\sum_{i \in I} s_{ik}x_i}{\sum_{i \in I} x_i} &\leq p_{ij} & \forall k \in S, \forall i \in I\\
    x_i &\geq 0 \quad \forall i \in I
\end{align}


## <font color='8EC044'> **Implementación del modelo** </font>

Creamos el objeto `ampl`

In [ ]:
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # Los optimizadores que vamos a usar
    license_uuid="default") # license to use (Aqui hay que poner su licencia :-;


Definimos una función que cree el modelo

In [ ]:
def create_model(ampl, ingredients, types_ing, nutrients, cost, contrib, min_nutri, max_nutri, pmix, ind_type):
  #Conjuntos
  ampl.eval(
    """ reset;
    set I;   # Conjunto de ingredientes
    set S;   # Conjunto de tipos de ingredientes
    set N;   # Conjunto de nutrientes

    param cost{i in I};     # Costo por kilogramo del ingrediente i
    param contrib{i in I, j in N};  # Aporte nutricional del ingrediente i al nutriente j
    param min_nutri{j in N};  # Cantidad mínima requerida del nutriente j
    param max_nutri{j in N};  # Cantidad máxima permitida del nutriente j
    param pmix{k in S};     # Proporción máxima de tipo de ingrediente k en la mezcla
    param ind_type{i in I, k in S};  # Indicador binario que define si el ingrediente i es del tipo k

    var x{i in I} >= 0;  # Cantidad de kilogramos del ingrediente i a incluir en el menú diario

    minimize Total_cost:
        sum{i in I} cost[i] * x[i];

    subject to min_nutrient{j in N}:
        sum{i in I} contrib[i, j] * x[i] >= min_nutri[j];

    subject to max_nutrient{j in N}:
        sum{i in I} contrib[i, j] * x[i] <= max_nutri[j];

    subject to mix_proportion{k in S}:
        sum{i in I} ind_type[i, k] * x[i] - sum{i in I} x[i]*pmix[k] <= 0;""")

  # instantiate the model
  ampl.set["I"] = ingredients
  ampl.set["S"] = types_ing
  ampl.set["N"] = nutrients
  ampl.param["cost"] = cost
  ampl.param["contrib"] = contrib
  ampl.param["min_nutri"] = min_nutri
  ampl.param["max_nutri"] = max_nutri
  ampl.param["pmix"] = pmix
  ampl.param["ind_type"] = ind_type

  return ampl

Consideramos la instancia de datos del modelo

In [ ]:
# Datos de ejemplo con nombres reales
ingredients = ["Pollo", "Res", "Pescado", "Tomate", "Brócoli", "Lechuga", "Arroz", "Frijol", "Lenteja", "Aguacate", "Mango", "Banano"]
types_ing = ["Carne", "Verduras", "Granos", "Frutas"]
nutrients = ['Proteína', 'Grasa', 'Carbohidratos', 'Fibra', 'Vitaminas']

# Costo por kilogramo del ingrediente por kilo (miles de pesos)
cost = {
    "Pollo": 15.6, "Res": 18.4, "Pescado": 19, "Tomate": 3.8, "Brócoli": 10.4, "Lechuga": 2.8,
    "Arroz": 5.1, "Frijol": 7.9, "Lenteja": 14.8, "Aguacate": 10.4, "Mango": 7.2, "Banano": 2.7
}

# Aporte nutricional del ingrediente al nutriente (en unidades por kg)
contrib = {
    ('Pollo', 'Proteína'): 25, ('Pollo', 'Grasa'): 10, ('Pollo', 'Carbohidratos'): 0, ('Pollo', 'Fibra'): 0, ('Pollo', 'Vitaminas'): 1,
    ('Res', 'Proteína'): 20, ('Res', 'Grasa'): 15, ('Res', 'Carbohidratos'): 0, ('Res', 'Fibra'): 0, ('Res', 'Vitaminas'): 1,
    ('Pescado', 'Proteína'): 22, ('Pescado', 'Grasa'): 8, ('Pescado', 'Carbohidratos'): 0, ('Pescado', 'Fibra'): 0, ('Pescado', 'Vitaminas'): 2,
    ('Tomate', 'Proteína'): 1, ('Tomate', 'Grasa'): 0, ('Tomate', 'Carbohidratos'): 5, ('Tomate', 'Fibra'): 2, ('Tomate', 'Vitaminas'): 15,
    ('Brócoli', 'Proteína'): 3, ('Brócoli', 'Grasa'): 0, ('Brócoli', 'Carbohidratos'): 5, ('Brócoli', 'Fibra'): 10, ('Brócoli', 'Vitaminas'): 20,
    ('Lechuga', 'Proteína'): 1, ('Lechuga', 'Grasa'): 0, ('Lechuga', 'Carbohidratos'): 2, ('Lechuga', 'Fibra'): 3, ('Lechuga', 'Vitaminas'): 10,
    ('Arroz', 'Proteína'): 6, ('Arroz', 'Grasa'): 1, ('Arroz', 'Carbohidratos'): 80, ('Arroz', 'Fibra'): 4, ('Arroz', 'Vitaminas'): 2,
    ('Frijol', 'Proteína'): 8, ('Frijol', 'Grasa'): 1, ('Frijol', 'Carbohidratos'): 60, ('Frijol', 'Fibra'): 8, ('Frijol', 'Vitaminas'): 3,
    ('Lenteja', 'Proteína'): 10, ('Lenteja', 'Grasa'): 1, ('Lenteja', 'Carbohidratos'): 50, ('Lenteja', 'Fibra'): 15, ('Lenteja', 'Vitaminas'): 4,
    ('Aguacate', 'Proteína'): 2, ('Aguacate', 'Grasa'): 15, ('Aguacate', 'Carbohidratos'): 8, ('Aguacate', 'Fibra'): 7, ('Aguacate', 'Vitaminas'): 10,
    ('Mango', 'Proteína'): 1, ('Mango', 'Grasa'): 0, ('Mango', 'Carbohidratos'): 20, ('Mango', 'Fibra'): 3, ('Mango', 'Vitaminas'): 30,
    ('Banano', 'Proteína'): 1, ('Banano', 'Grasa'): 0, ('Banano', 'Carbohidratos'): 30, ('Banano', 'Fibra'): 2, ('Banano', 'Vitaminas'): 20
}


# Cantidad mínima requerida del nutriente (en unidades)
min_nutri = {'Proteína': 50, 'Grasa': 30, 'Carbohidratos': 50, 'Fibra': 25, 'Vitaminas': 10}

# Cantidad máxima permitida del nutriente (en unidades)
max_nutri = {'Proteína': 150, 'Grasa': 50, 'Carbohidratos': 200, 'Fibra': 50, 'Vitaminas': 30}

# Proporción máxima de tipo de ingrediente en la mezcla
pmix = {"Carne": 0.4, "Verduras": 0.3, "Granos": 0.2, "Frutas": 0.5}

# Indicador binario que define si el ingrediente es del tipo k
ind_type = {(ing, ing_type):0 for ing in ingredients for ing_type in types_ing}
ind_type_sparce = {
    ("Pollo", "Carne"): 1, ("Res", "Carne"): 1, ("Pescado", "Carne"): 1,
    ("Tomate", "Verduras"): 1, ("Brócoli", "Verduras"): 1, ("Lechuga", "Verduras"): 1,
    ("Arroz", "Granos"): 1, ("Frijol", "Granos"): 1, ("Lenteja", "Granos"): 1,
    ("Aguacate", "Frutas"): 1, ("Mango", "Frutas"): 1, ("Banano", "Frutas"): 1}
ind_type.update(ind_type_sparce)



Definimos algunos parámetros para el optimizador

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

Creamos el modelo, resolvemos e imprimimos la solución

In [ ]:
ampl = create_model(ampl, ingredients, types_ing, nutrients, cost, contrib, min_nutri, max_nutri, pmix, ind_type)
ampl.solve()
# Imprimimos los resultados
Total_cost = ampl.get_objective("Total_cost")
print("\nObjective is:", Total_cost.get().value())
ampl.get_data("x").to_pandas()

## <font color='8EC044'> **Análisis de sensibilidady dualidad** </font>

Desarrolle el análisis de sensibilidad y dualidad respecto a los parámetros del modelo interpretando sus resultados de acuerdo al contexto del problema

# <font color='056938'> **Referencias** </font>

[1] Tovey, C. A. (2020). Linear Optimization and Duality: A Modern Exposition. Chapman and Hall/CRC.

[2] Bazaraa, M. S., Jarvis, J. J., & Sherali, H. D. (2011). Linear programming and network flows. John Wiley & Sons.